# 01. Conhecendo a Requests


In [1]:
import requests as req


In [2]:
res = req.get('https://api.github.com/events')
res


<Response [200]>

## Explorando a biblioteca

Essa biblioteca é usada para fazer requisições HTTP, então tudo o que usamos em requisições Web comuns funcionam aqui!

Isso é algo importante de se lembrar pois existem alguns padrões como Methods, Status Code e padrões de requisições que se mantem e existem nessa lib.

In [3]:
res.status_code


200

In [4]:
res.url


'https://api.github.com/events'

In [5]:
print(f'Tipo do json response (para esse caso): {type(res.json())}')
print('Amostra de um item nessa lista:')
res.json()[0]


Tipo do json response (para esse caso): <class 'list'>
Amostra de um item nessa lista:


{'id': '42144356434',
 'type': 'PushEvent',
 'actor': {'id': 74199667,
  'login': 'Chroiud',
  'display_login': 'Chroiud',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/Chroiud',
  'avatar_url': 'https://avatars.githubusercontent.com/u/74199667?'},
 'repo': {'id': 821590782,
  'name': 'Chroiud/playercount',
  'url': 'https://api.github.com/repos/Chroiud/playercount'},
 'payload': {'repository_id': 821590782,
  'push_id': 20348405080,
  'size': 1,
  'distinct_size': 1,
  'ref': 'refs/heads/main',
  'head': 'e97dbd40b0171d0dba45ce8e1c79deec1259cb3d',
  'before': 'd00175ae817259c7f0a56b2351feafa7e962210a',
  'commits': [{'sha': 'e97dbd40b0171d0dba45ce8e1c79deec1259cb3d',
    'author': {'email': '74199667+Chroiud@users.noreply.github.com',
     'name': 'Chroiud'},
    'message': 'Update player count',
    'distinct': True,
    'url': 'https://api.github.com/repos/Chroiud/playercount/commits/e97dbd40b0171d0dba45ce8e1c79deec1259cb3d'}]},
 'public': True,
 'created_at': '2024-09

## Usando outras URLs

Quando queremos acessar dados diferentes em uma API alteramos a URL que estamos utilizando na requisição. Sendo que é necessário ter uma ideia da documentação da API para saber quais são os dados mais interessantes a serem utilizados.

Outro ponto interessante é se atentar a versão da API, uma API pode ter diferentes versões e isso pode impactar no uso de seus endpoints!

In [6]:
res = req.get('https://api.github.com/versions')
res.status_code


200

In [7]:
res.json()


['2022-11-28']

Por uma questão de segurança e manutenção de código iremos usar por padrão essa versão da API (2022-11-28), só iremos alterar no futuro a nível de manutençao do projeto.

Para isso usamos os headers, que são configurações adicionais que passamos junto com a request.

# 02. Extraindo os dados

## Obtendo dados com GET

In [8]:
# especificando a versão da API
headers = {'X-GitHub-Api-Version': '2022-11-28'}


In [9]:
# definicao da api de requisicao dos repos da Amazon
api_base_url = 'https://api.github.com'
owner = 'amzn'
url = f'{api_base_url}/users/{owner}/repos'


In [10]:
response = req.get(url, headers)
response.status_code


200

In [11]:
amzn_repos = response.json()
len(amzn_repos)


30

Só conseguimos acessar 30 repos pois esse endpoint tem paginação. Para conseguirmos acessar mais repos precisamos nos autenticar. Esse é um requisito dessa API.

## Autenticação

Solicitações autenticadas têm um limite de taxa mais alto. Quando um usuário faz uma solicitação autenticada, ele fornece credenciais que comprovam sua identidade, o que permite que a API confie nele e lhe conceda acesso a recursos e funcionalidades adicionais.

Além disso, a maioria das APIs estabelece limites para o número de solicitações que um usuário pode fazer em um determinado período de tempo, conhecido como 'limite de taxa'. Quando um usuário faz solicitações autenticadas, a API geralmente permite que ele faça mais solicitações em um determinado período de tempo, devido à maior confiança e credibilidade que a autenticação fornece.

In [12]:
%pip install python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [13]:
from os import getenv
from dotenv import load_dotenv

load_dotenv()

access_token = getenv('GITHUB_ACCESS_TOKEN')
headers['Authorization'] = f'Bearer {access_token}'


## Paginando os repositórios

In [15]:
response = req.get(url, headers)


In [16]:
api_base_url = 'https://api.github.com'
owner = 'amzn'
url = f'{api_base_url}/users/{owner}/repos'


In [17]:
repos_list = []
for page_index in range(1,7):
    try:
        url_page = f'{url}?page={page_index}'
        repos = req.get(url_page, headers).json()
        
        repos_list.append(repos)
    except:
        print('Error while getting repos')
        repos_list.append(None)


repos_list

# 03. Transformando os dados

## Nomes dos repositórios

In [18]:
repos_list[0][2]['name']


'ads-pao-amznjs-gtm-template'

In [19]:
# repos_name = []

# for page in repos_list:
#     for repo in page:
#         repos_name.append(repo['name'])

repos_name = [repo['name'] for repos in repos_list for repo in repos]

print('Total de repos:', len(repos_name))
repos_name[:10]


Total de repos: 159


['.github',
 'ads-advanced-tools-docs',
 'ads-pao-amznjs-gtm-template',
 'alexa-coho',
 'alexa-skills-kit-js',
 'amazon-ads-advertiser-audience-normalization-sdk-py',
 'amazon-advertising-api-php-sdk',
 'amazon-codeguru-profiler-for-spark',
 'amazon-frustration-free-setup-certification-tool',
 'amazon-hub-counter-api-docs']

## Linguagens utilizadas

In [20]:
repos_list[0][2]['language']


'Smarty'

In [21]:
repos_language = [repo['language'] for repos in repos_list for repo in repos]

repos_language[:10]


[None,
 'Jupyter Notebook',
 'Smarty',
 'JavaScript',
 None,
 'Python',
 'PHP',
 'Java',
 'Python',
 'CSS']

## Criando um DataFrame

In [22]:
import pandas as pd


In [23]:
dados_amzn = pd.DataFrame()

dados_amzn['repository_name'] = repos_name
dados_amzn['language'] = repos_language

dados_amzn.head()


,repository_name,language
0,.github,None
1,ads-advanced-tools-docs,Jupyter Notebook
2,ads-pao-amznjs-gtm-template,Smarty
3,alexa-coho,JavaScript
4,alexa-skills-kit-js,None


In [24]:
dados_amzn.to_csv('../data_processed/amazon_repos.csv', index=False) 
# index=False foi usado para que o indice nao seja salvo


# 04. Armazenando os dados


## Criando repositório com POST


In [25]:
url = f'{api_base_url}/user/repos'

url


'https://api.github.com/user/repos'

In [28]:
data = {
    'name': 'linguagens-utilizadas',
    'description': 'Repositorio com as linguagens de prog. da Amazon',
    'private': True
}

request = req.post(url, json=data, headers=headers)
print('Status code:', request.status_code)
request.json()


Status code: 201


{'id': 861027843,
 'node_id': 'R_kgDOM1I-Aw',
 'name': 'linguagens-utilizadas',
 'full_name': 'juanveronez/linguagens-utilizadas',
 'private': True,
 'owner': {'login': 'juanveronez',
  'id': 37226530,
  'node_id': 'MDQ6VXNlcjM3MjI2NTMw',
  'avatar_url': 'https://avatars.githubusercontent.com/u/37226530?v=4',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/juanveronez',
  'html_url': 'https://github.com/juanveronez',
  'followers_url': 'https://api.github.com/users/juanveronez/followers',
  'following_url': 'https://api.github.com/users/juanveronez/following{/other_user}',
  'gists_url': 'https://api.github.com/users/juanveronez/gists{/gist_id}',
  'starred_url': 'https://api.github.com/users/juanveronez/starred{/owner}{/repo}',
  'subscriptions_url': 'https://api.github.com/users/juanveronez/subscriptions',
  'organizations_url': 'https://api.github.com/users/juanveronez/orgs',
  'repos_url': 'https://api.github.com/users/juanveronez/repos',
  'events_url': 'https://api.gi